# KE XUE CHAO GU

In [2]:
import re
import json
import csv
from io import StringIO
from bs4 import BeautifulSoup as bs
import requests
import pyspark 
import pandas as pd
import datetime as dt
import plotly.graph_objects as go

In [10]:
stock='FB'

In [11]:
url_stats= 'https://finance.yahoo.com/quote/{}/key-statistics?p={}'
url_profile='https://finance.yahoo.com/quote/{}/profile?p={}'
url_statement= 'https://finance.yahoo.com/quote/{}/financials?p={}'

In [12]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
response = requests.get(url_statement.format(stock,stock),headers=headers)
soup= bs(response.text,'html.parser')
pattern=re.compile(r'\s--\sData\s--\s')
script_data=soup.find('script',text=pattern).contents[0]
start=script_data.find("context")-2
json_data=json.loads(script_data[start:-12])

In [13]:
annual_is=json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']
quarterly_is= json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistoryQuarterly']['incomeStatementHistory']


annaul_cf=json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistory']['cashflowStatements']
quarterly_cf=json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistoryQuarterly']['cashflowStatements']


annaul_bs=json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
quarterly_bs=json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistoryQuarterly']['balanceSheetStatements']

# INCOME STATEMENTS

In [14]:
annual_is_stmts =[]
#consolidating annual
for s in annual_is:
    statement ={}
    for key, val in s.items():
        try:
            statement[key]=val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    annual_is_stmts.append(statement)

In [15]:
annual_is_df = pd.DataFrame(annual_is_stmts)  
annual_is_df ['endDate'] = annual_is_df ['endDate'].apply(lambda x: dt.date.fromtimestamp(x).strftime('%Y%m%d'))
first_column = annual_is_df.pop('endDate')
annual_is_df.insert(0, 'endDate', first_column)
#annual_is_df

In [16]:
annual_is_df_display=annual_is_df.T
new_header = annual_is_df_display.iloc[0] #grab the first row for the header
annual_is_df_display = annual_is_df_display[1:] #take the data less the header row
annual_is_df_display.columns = new_header #set the header row as the df header
annual_is_df_display

endDate,20201230,20191230,20181230,20171230
researchDevelopment,18447000000,13600000000,10273000000,7754000000
incomeBeforeTax,33180000000,24812000000,25361000000,20594000000
netIncome,29146000000,18485000000,22112000000,15934000000
sellingGeneralAdministrative,18155000000,15341000000,11297000000,7242000000
grossProfit,69273000000,57927000000,46483000000,35199000000
ebit,32671000000,28986000000,24913000000,20203000000
operatingIncome,32671000000,28986000000,24913000000,20203000000
incomeTaxExpense,4034000000,6327000000,3249000000,4660000000
totalRevenue,85965000000,70697000000,55838000000,40653000000
totalOperatingExpenses,53294000000,41711000000,30925000000,20450000000


In [17]:
quarter_is_stmts =[]
#consolidating annual
for s in quarterly_is:
    statement ={}
    for key, val in s.items():
        try:
            statement[key]=val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    quarter_is_stmts.append(statement)

In [18]:
quarter_is_df = pd.DataFrame(quarter_is_stmts)  
quarter_is_df ['endDate'] = quarter_is_df ['endDate'].apply(lambda x: dt.date.fromtimestamp(x).strftime('%Y%m%d'))
first_column = quarter_is_df.pop('endDate')
quarter_is_df.insert(0, 'endDate', first_column)
#quarter_is_df

In [19]:
df_qurteris_display=quarter_is_df.T
new_header = df_qurteris_display.iloc[0] #grab the first row for the header
df_qurteris_display = df_qurteris_display[1:] #take the data less the header row
df_qurteris_display.columns = new_header #set the header row as the df header
df_qurteris_display

endDate,20210929,20210629,20210330,20201230
researchDevelopment,6316000000,6096000000,5197000000,5207000000
incomeBeforeTax,10565000000,12513000000,11503000000,13055000000
netIncome,9194000000,10394000000,9497000000,11219000000
sellingGeneralAdministrative,6500000000,5215000000,4465000000,4880000000
grossProfit,23239000000,23678000000,21040000000,22862000000
ebit,10423000000,12367000000,11378000000,12775000000
operatingIncome,10423000000,12367000000,11378000000,12775000000
incomeTaxExpense,1371000000,2119000000,2006000000,1836000000
totalRevenue,29010000000,29077000000,26171000000,28072000000
totalOperatingExpenses,18587000000,16710000000,14793000000,15297000000


# BALENCE SHEET

In [20]:
## annual bs
annual_bs_stmts =[]
for s in annaul_bs:
    statement ={}
    for key, val in s.items():
        try:
            statement[key]=val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    annual_bs_stmts.append(statement)
    
    
##quarter bs
quarter_bs_stmts =[]
for s in quarterly_bs:
    statement ={}
    for key, val in s.items():
        try:
            statement[key]=val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    quarter_bs_stmts.append(statement)
    
    
##annual_bs_transformation
annual_bs_df = pd.DataFrame(annual_bs_stmts)  
annual_bs_df['endDate'] = annual_bs_df ['endDate'].apply(lambda x: dt.date.fromtimestamp(x).strftime('%Y%m%d'))
first_column = annual_bs_df.pop('endDate')
annual_bs_df.insert(0, 'endDate', first_column)

##Quarterly_bs_transformation
quarter_bs_df = pd.DataFrame(quarter_bs_stmts)  
quarter_bs_df['endDate'] = quarter_bs_df ['endDate'].apply(lambda x: dt.date.fromtimestamp(x).strftime('%Y%m%d'))
first_column = quarter_bs_df.pop('endDate')
quarter_bs_df.insert(0, 'endDate', first_column)

In [21]:
##annual_bs_display
df_annualbs_display=annual_bs_df.T
new_header = df_annualbs_display.iloc[0] #grab the first row for the header
df_annualbs_display = df_annualbs_display[1:] #take the data less the header row
df_annualbs_display.columns = new_header #set the header row as the df header
df_annualbs_display

endDate,20201230,20191230,20181230,20171230
intangibleAssets,623000000,894000000,1294000000,1884000000
capitalSurplus,50018000000,45851000000,42906000000,40584000000
totalLiab,31026000000,32322000000,13207000000,10177000000
totalStockholderEquity,128290000000,101054000000,84127000000,74347000000
otherCurrentLiab,9964000000,11186000000,4494000000,2590000000
totalAssets,159316000000,133376000000,97334000000,84524000000
otherCurrentAssets,241000000,8000000,10000000,18000000
retainedEarnings,77345000000,55692000000,41981000000,33990000000
otherLiab,5945000000,7745000000,6190000000,6417000000
goodWill,19050000000,18715000000,18301000000,18221000000


In [22]:
##quarter_bs_display
df_quarterbs_display=quarter_bs_df.T
new_header = df_quarterbs_display.iloc[0] #grab the first row for the header
df_quarterbs_display = df_quarterbs_display[1:] #take the data less the header row
df_quarterbs_display.columns = new_header #set the header row as the df header
df_quarterbs_display

endDate,20210929,20210629,20210330,20201230
intangibleAssets,365000000,514000000,505000000,623000000
capitalSurplus,54334000000,52845000000,51160000000,50018000000
totalLiab,36225000000,32382000000,29866000000,31026000000
totalStockholderEquity,133360000000,138227000000,133657000000,128290000000
otherCurrentLiab,1373000000,1340000000,1388000000,9964000000
totalAssets,169585000000,170609000000,163523000000,159316000000
otherCurrentAssets,195000000,201000000,257000000,241000000
retainedEarnings,79233000000,85097000000,82343000000,77345000000
otherLiab,6352000000,6059000000,6101000000,5945000000
goodWill,19065000000,19219000000,19056000000,19050000000


# CASH FlOW STATEMENT

In [23]:
## annual CF
annual_cf_stmts =[]
for s in annaul_cf:
    statement ={}
    for key, val in s.items():
        try:
            statement[key]=val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    annual_cf_stmts.append(statement)
    
    
##quarter bs
quarter_cf_stmts =[]
for s in quarterly_cf:
    statement ={}
    for key, val in s.items():
        try:
            statement[key]=val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    quarter_cf_stmts.append(statement)
    
    
##annual_cf_transformation
annual_cf_df = pd.DataFrame(annual_cf_stmts)  
annual_cf_df['endDate'] = annual_cf_df ['endDate'].apply(lambda x: dt.date.fromtimestamp(x).strftime('%Y%m%d'))
first_column = annual_cf_df.pop('endDate')
annual_cf_df.insert(0, 'endDate', first_column)


##Quarterly_cf_transformation
quarter_cf_df = pd.DataFrame(quarter_cf_stmts)  
quarter_cf_df['endDate'] = quarter_cf_df ['endDate'].apply(lambda x: dt.date.fromtimestamp(x).strftime('%Y%m%d'))
first_column = quarter_cf_df.pop('endDate')
quarter_cf_df.insert(0, 'endDate', first_column)

In [24]:
##annual_cf_display
df_annualcf_display=annual_cf_df.T
new_header = df_annualcf_display.iloc[0] 
df_annualcf_display = df_annualcf_display[1:]
df_annualcf_display.columns = new_header 
df_annualcf_display

endDate,20201230,20191230,20181230,20171230
investments,-14520000000,-4254000000,2449000000,-13250000000
changeToLiabilities,91000000,236000000,274000000,47000000
totalCashflowsFromInvestingActivities,-30059000000,-19864000000,-11603000000,-20118000000
netBorrowings,-580000000,-775000000,500000000,500000000
totalCashFromFinancingActivities,-10292000000,-7299000000,-15572000000,-5235000000
changeToOperatingActivities,-1302000000,8975000000,91000000,3449000000
netIncome,29146000000,18485000000,22112000000,15934000000
changeInCash,-1325000000,9155000000,1920000000,-905000000
repurchaseOfStock,-9836000000,-6539000000,-16087000000,-5222000000
effectOfExchangeRate,279000000,4000000,-179000000,232000000


In [25]:
##quarter_cf_display
df_quartercf_display=quarter_cf_df.T
new_header = df_quartercf_display.iloc[0] #grab the first row for the header
df_quartercf_display = df_quartercf_display[1:] #take the data less the header row
df_quartercf_display.columns = new_header #set the header row as the df header
df_quartercf_display

endDate,20210929,20210629,20210330,20201230
investments,4153000000,-3264000000,-600000000,-529000000
changeToLiabilities,772000000,119000000,-244000000,-59000000
totalCashflowsFromInvestingActivities,-330000000,-8195000000,-4874000000,-5173000000
netBorrowings,-219000000,-70000000,-201000000,-158000000
totalCashFromFinancingActivities,-15252000000,-8549000000,-5185000000,-3207000000
changeToOperatingActivities,1166000000,-643000000,-2014000000,2606000000
netIncome,9194000000,10394000000,9497000000,11219000000
changeInCash,-1707000000,-3380000000,1937000000,5975000000
repurchaseOfStock,-15033000000,-8434000000,-5016000000,-3049000000
effectOfExchangeRate,-215000000,117000000,-246000000,315000000


# Stock Price

In [8]:
 def get_stock(stockindex,howlong,interval):
        
#confingure user so yahoo knows it's not a bot <_<
        headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

        stock_url='https://query1.finance.yahoo.com/v7/finance/download/{}?'.format(stockindex)
#confingure parameters so user could use it to determine what they what
        params={
          'range':'{}'.format(howlong),
          'interval':'{}'.format(interval),
          'events':'history',
          'includeAdjustedClose':'true'}
# PULLL THE FUCKINGG DATA !!!!!!!!!
        response = requests.get(stock_url,headers=headers,params=params)
#convert to list then df
        file=StringIO(response.text)
        reader=csv.reader(file)
        stock_data=list(reader)
        stock_df=pd.DataFrame(stock_data)
#now transform :)
        new_header = stock_df.iloc[0]
        stock_df = stock_df[1:]
        stock_df.columns = new_header
        return stock_df

In [9]:
#STOCK CHOICES: any stock as long as you know the short name for it......
#range choices: 1d, 5d,1m,3m,6m,1y,2y,5y
#interval chioce: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
get_stock('FB','1y','1d')


,Date,Open,High,Low,Close,Adj Close,Volume
1,2020-10-29,276.549988,283.600006,273.779999,280.829987,280.829987,32368100
2,2020-10-30,274.500000,276.700012,259.100006,263.109985,263.109985,47247800
3,2020-11-02,264.600006,270.470001,257.339996,261.359985,261.359985,27165700
4,2020-11-03,263.160004,270.049988,260.880005,265.299988,265.299988,17961900
5,2020-11-04,281.000000,288.940002,278.619995,287.380005,287.380005,35364400
...,...,...,...,...,...,...,...
248,2021-10-22,326.350006,329.630005,321.109985,324.609985,324.609985,35152700
249,2021-10-25,320.299988,329.559998,319.720001,328.690002,328.690002,38409000
250,2021-10-26,328.260010,330.209991,309.600006,315.809998,315.809998,65654000
251,2021-10-27,314.190002,319.250000,312.059998,312.220001,312.220001,29971800
